In [1]:
import random

# Definición de capacidades de tolvas y niveles iniciales
tolvas_MC1 = {
    "Clinker": {"capacidad": 500, "max_metros": 14},
    "Puzolana": {"capacidad": 300, "max_metros": 12},
    "Yeso": {"capacidad": 300, "max_metros": 10}
}
tolvas_MC2 = {
    "Clinker": {"capacidad": 300, "max_metros": 9},
    "Puzolana_Humeda": {"capacidad": 500, "max_metros": 15, "tolva": "426HO04"},
    "Puzolana_Seca": {"capacidad": 100, "max_metros": 12, "tolva": "426HO02"},
    "Yeso": {"capacidad": 120, "max_metros": 9}
}
tolvas_MC3 = {
    "Clinker": {"capacidad": 60, "max_porcentaje": 100},
    "Clinker_Silo_Blanco": {"capacidad": 500, "max_metros": 10.5},
    "Puzolana": {"capacidad": 35, "max_porcentaje": 100},
    "Yeso": {"capacidad": 30, "max_porcentaje": 100}
}

niveles_MC1 = {"Clinker": 0, "Puzolana": 0, "Yeso": 0}
niveles_MC2 = {"Clinker": 0, "Puzolana_Humeda": 0, "Puzolana_Seca": 0, "Yeso": 0}
niveles_MC3 = {"Clinker": 0, "Clinker_Silo_Blanco": 0, "Puzolana": 0, "Yeso": 0}

# Variable global para rastrear alimentaciones actuales
alimentaciones_actuales = set()

def resetear_niveles_molino(molino):
    global niveles_MC1, niveles_MC2, niveles_MC3
    if molino == "MC1":
        niveles_MC1 = {"Clinker": 0, "Puzolana": 0, "Yeso": 0}
    elif molino == "MC2":
        niveles_MC2 = {"Clinker": 0, "Puzolana_Humeda": 0, "Puzolana_Seca": 0, "Yeso": 0}
    elif molino == "MC3":
        niveles_MC3 = {"Clinker": 0, "Clinker_Silo_Blanco": 0, "Puzolana": 0, "Yeso": 0}

def ajustar_velocidad(material, min_vel, max_vel):
    while True:
        try:
            velocidad = float(input(f"Ingrese la velocidad del transportador para {material} ({min_vel}-{max_vel} RPM): "))
            if min_vel <= velocidad <= max_vel:
                print(f"Ajustando velocidad del transportador a {velocidad:.2f} RPM para {material}")
                return velocidad
            else:
                print(f"La velocidad debe estar entre {min_vel} y {max_vel} RPM.")
        except ValueError:
            print("Por favor, ingrese un número válido.")

def verificar_nivel_yeso(molino, nivel_actual):
    if molino == "MC1" and nivel_actual > 2:
        print(f"Advertencia: El nivel de yeso en {molino} ({nivel_actual:.2f}m) excede el límite de 2m.")
        while True:
            try:
                nuevo_nivel = float(input("Ingrese el nuevo nivel de yeso (debe ser 2m o menos): "))
                if nuevo_nivel <= 2:
                    return nuevo_nivel
                else:
                    print("El nivel debe ser 2m o menos.")
            except ValueError:
                print("Por favor, ingrese un número válido.")
    return nivel_actual

def actualizar_alimentaciones(molino, material, accion):
    global alimentaciones_actuales
    if accion == "agregar":
        alimentaciones_actuales.add((molino, material))
    elif accion == "remover":
        alimentaciones_actuales.discard((molino, material))

def validar_restricciones(molino, material):
    global alimentaciones_actuales

    if molino == "MC1" and material in ["Puzolana", "Yeso"]:
        if ("MC1", "Puzolana") in alimentaciones_actuales or ("MC1", "Yeso") in alimentaciones_actuales:
            return False, "No se puede alimentar Puzolana y Yeso simultáneamente a MC1"

    if molino == "MC2" and material in ["Yeso", "Puzolana_Humeda", "Puzolana_Seca"]:
        conflictos = [("MC2", "Yeso"), ("MC2", "Puzolana_Humeda"), ("MC2", "Puzolana_Seca"),
                      ("MC1", "Yeso"), ("MC1", "Puzolana"), ("MC3", "Yeso")]
        if any(conflicto in alimentaciones_actuales for conflicto in conflictos):
            return False, "Conflicto en alimentación simultánea para MC2"

    if molino == "MC3":
        if any(alim[0] == "MC3" for alim in alimentaciones_actuales):
            return False, "Solo se puede alimentar un material a la vez a MC3"

    if material == "Clinker":
        if any(alim[1] == "Clinker" for alim in alimentaciones_actuales):
            return False, "No se puede alimentar Clinker a múltiples molinos simultáneamente"

    return True, "Validación exitosa"

def intentar_alimentar(molino, material, cantidad):
    global niveles_MC1, niveles_MC2, niveles_MC3

    valido, mensaje = validar_restricciones(molino, material)
    if not valido:
        print(f"No se puede alimentar {material} a {molino}: {mensaje}")
        return 0

    actualizar_alimentaciones(molino, material, "agregar")

    if molino == "MC1":
        niveles = niveles_MC1
        tolvas = tolvas_MC1
    elif molino == "MC2":
        niveles = niveles_MC2
        tolvas = tolvas_MC2
    elif molino == "MC3":
        niveles = niveles_MC3
        tolvas = tolvas_MC3

    if material in ["Puzolana", "Puzolana_Humeda", "Puzolana_Seca"]:
        ajustar_velocidad("puzolana", 1100, 1150)
    elif material == "Yeso":
        ajustar_velocidad("yeso", 850, 900)

    capacidad = tolvas[material]['capacidad']
    if 'max_metros' in tolvas[material]:
        max_nivel = tolvas[material]['max_metros']
        nivel_actual = niveles[material] / capacidad * max_nivel
        if molino == "MC1" and material == "Yeso":
            max_nivel = min(max_nivel, 2)  # Limitar a 2m para yeso en MC1
        espacio_disponible = max_nivel - nivel_actual
        cantidad_metros = min(cantidad / capacidad * max_nivel, espacio_disponible)
        cantidad_alimentada = cantidad_metros / max_nivel * capacidad
    else:  # Para MC3 que usa porcentajes
        max_nivel = tolvas[material]['max_porcentaje']
        nivel_actual = niveles[material] / capacidad * 100
        espacio_disponible = max_nivel - nivel_actual
        cantidad_porcentaje = min(cantidad / capacidad * 100, espacio_disponible)
        cantidad_alimentada = cantidad_porcentaje / 100 * capacidad

    if cantidad_alimentada > 0:
        niveles[material] += cantidad_alimentada
        if 'max_metros' in tolvas[material]:
            print(f"Alimentado {cantidad_alimentada:.2f}t de {material} a {molino} (Nivel: {nivel_actual + cantidad_metros:.2f}m)")
        else:
            print(f"Alimentado {cantidad_alimentada:.2f}t de {material} a {molino} (Nivel: {nivel_actual + cantidad_porcentaje:.2f}%)")
    else:
        print(f"No se pudo alimentar {material} a {molino}. Tolva llena o se alcanzó el límite máximo.")

    actualizar_alimentaciones(molino, material, "remover")
    return cantidad_alimentada

def todas_tolvas_llenas():
    if any(nivel < tolvas_MC1[material]['capacidad'] * 0.8 for material, nivel in niveles_MC1.items()):
        return False
    if any(nivel < tolvas_MC2[material]['capacidad'] * 0.8 for material, nivel in niveles_MC2.items()):
        return False
    if any(nivel < tolvas_MC3[material]['capacidad'] * 0.5 for material, nivel in niveles_MC3.items()):
        return False
    return True

def calcular_necesidades(molino, tipo_produccion):
    produccion_MC1 = {
        "P30": (0.30, 0.015, 72),
        "P40": (0.40, 0.015, 64)
    }
    produccion_MC2 = {
        "P10": (0.10, 0.03, 70),
        "P16": (0.16, 0.025, 80),
        "P20": (0.12, 0.025, 87),
        "P30": (0.30, 0.02, 110)
    }
    produccion_MC3 = {
        "P30": (0.30, 0.025, 37)
    }

    if molino == "MC1" and tipo_produccion in produccion_MC1:
        puzolana, yeso, produccion = produccion_MC1[tipo_produccion]
        clinker = 1 - puzolana - yeso
        return {
            "Clinker": clinker * produccion,
            "Puzolana": puzolana * produccion,
            "Yeso": yeso * produccion
        }
    elif molino == "MC2" and tipo_produccion in produccion_MC2:
        puzolana, yeso, produccion = produccion_MC2[tipo_produccion]
        clinker = 1 - puzolana - yeso
        return {
            "Clinker": clinker * produccion,
            "Puzolana_Humeda": puzolana * produccion * 0.7,
            "Puzolana_Seca": puzolana * produccion * 0.3,
            "Yeso": yeso * produccion
        }
    elif molino == "MC3" and tipo_produccion in produccion_MC3:
        puzolana, yeso, produccion = produccion_MC3[tipo_produccion]
        clinker = 1 - puzolana - yeso
        return {
            "Clinker": clinker * produccion,
            "Puzolana": puzolana * produccion,
            "Yeso": yeso * produccion
        }

    print(f"Advertencia: No se encontró configuración para {molino} con producto {tipo_produccion}")
    return {"Clinker": 60, "Puzolana": 30, "Yeso": 10}

def redondear_diccionario(d, decimales=2):
    return {k: round(v, decimales) if isinstance(v, (int, float)) else v for k, v in d.items()}

def calcular_tiempos_llenado(molino, necesidades):
    tiempos_llenado = {}
    if molino == "MC1":
        rendimiento = 0.8
        for mat, cant in necesidades.items():
            if mat == "Yeso":
                capacidad_objetivo = min(tolvas_MC1[mat]["capacidad"] * rendimiento, 2 / tolvas_MC1[mat]["max_metros"] * tolvas_MC1[mat]["capacidad"])
            else:
                capacidad_objetivo = tolvas_MC1[mat]["capacidad"] * rendimiento
            nivel_actual = niveles_MC1[mat]
            if cant > 0:
                tiempos_llenado[mat] = (capacidad_objetivo - nivel_actual) / cant
            else:
                tiempos_llenado[mat] = float('inf')

    elif molino == "MC2":
        rendimiento = 0.8
        for mat, cant in necesidades.items():
            capacidad_objetivo = tolvas_MC2[mat]["capacidad"] * rendimiento
            nivel_actual = niveles_MC2[mat]
            if cant > 0:
                tiempos_llenado[mat] = (capacidad_objetivo - nivel_actual) / cant
            else:
                tiempos_llenado[mat] = float('inf')
    else:  # MC3
        rendimiento = 0.5
        for mat, cant in necesidades.items():
            capacidad_objetivo = tolvas_MC3[mat]["capacidad"] * rendimiento
            nivel_actual = niveles_MC3[mat]
            if cant > 0:
                tiempos_llenado[mat] = (capacidad_objetivo - nivel_actual) / cant
            else:
                tiempos_llenado[mat] = float('inf')

    return tiempos_llenado


def generar_recomendaciones(molino_actual):
    recomendaciones = []
    restricciones = []

    def espacio_disponible(nivel_actual, capacidad, rendimiento):
        return capacidad * rendimiento - nivel_actual

    if molino_actual == "MC1":
        rendimiento = 0.8
        esp_clinker = espacio_disponible(niveles_MC1["Clinker"], tolvas_MC1["Clinker"]["capacidad"], rendimiento)
        esp_puzolana = espacio_disponible(niveles_MC1["Puzolana"], tolvas_MC1["Puzolana"]["capacidad"], rendimiento)
        esp_yeso = min(espacio_disponible(niveles_MC1["Yeso"], tolvas_MC1["Yeso"]["capacidad"], rendimiento), 60 - niveles_MC1["Yeso"])

        if esp_clinker > 0:
            recomendaciones.append(f"Recomiendo alimentar Clinker a MC1 desde Pretrit (Espacio disponible: {esp_clinker:.2f}t)")
        if esp_puzolana > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana a MC1 (Espacio disponible: {esp_puzolana:.2f}t)")
        if esp_yeso > 0:
            recomendaciones.append(f"Recomiendo alimentar Yeso a MC1 (Espacio disponible: {esp_yeso:.2f}t)")

    elif molino_actual == "MC2":
        rendimiento = 0.8
        esp_clinker = espacio_disponible(niveles_MC2["Clinker"], tolvas_MC2["Clinker"]["capacidad"], rendimiento)
        esp_puzolana_h = espacio_disponible(niveles_MC2["Puzolana_Humeda"], tolvas_MC2["Puzolana_Humeda"]["capacidad"], rendimiento)
        esp_puzolana_s = espacio_disponible(niveles_MC2["Puzolana_Seca"], tolvas_MC2["Puzolana_Seca"]["capacidad"], rendimiento)
        esp_yeso = espacio_disponible(niveles_MC2["Yeso"], tolvas_MC2["Yeso"]["capacidad"], rendimiento)

        if esp_clinker > 0:
            recomendaciones.append(f"Recomiendo alimentar Clinker a MC2 desde Pretrit (Espacio disponible: {esp_clinker:.2f}t)")
        if esp_puzolana_h > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana Húmeda a 426HO04 por MC2 (Espacio disponible: {esp_puzolana_h:.2f}t)")
        if esp_puzolana_s > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana Seca a 426HO02 por 426HO04 (Espacio disponible: {esp_puzolana_s:.2f}t)")
        if esp_yeso > 0:
            recomendaciones.append(f"Recomiendo alimentar Yeso a MC2 por MC2 (Espacio disponible: {esp_yeso:.2f}t)")

    elif molino_actual == "MC3":
        rendimiento = 0.5
        esp_clinker = espacio_disponible(niveles_MC3["Clinker"], tolvas_MC3["Clinker"]["capacidad"], rendimiento)
        esp_clinker_sb = espacio_disponible(niveles_MC3["Clinker_Silo_Blanco"], tolvas_MC3["Clinker_Silo_Blanco"]["capacidad"], rendimiento)
        esp_puzolana = espacio_disponible(niveles_MC3["Puzolana"], tolvas_MC3["Puzolana"]["capacidad"], rendimiento)
        esp_yeso = espacio_disponible(niveles_MC3["Yeso"], tolvas_MC3["Yeso"]["capacidad"], rendimiento)

        if esp_clinker > 0:
            if niveles_MC3["Clinker_Silo_Blanco"] / tolvas_MC3["Clinker_Silo_Blanco"]["capacidad"] > 0.3:
                recomendaciones.append(f"Recomiendo alimentar Clinker a MC3 desde Silo Blanco (Espacio disponible: {esp_clinker:.2f}t)")
            else:
                recomendaciones.append(f"Recomiendo alimentar Clinker a Silo Blanco desde Pretrit (Espacio disponible: {esp_clinker_sb:.2f}t)")
        if esp_puzolana > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana Seca a MC3 por MC2 (Espacio disponible: {esp_puzolana:.2f}t)")
        if esp_yeso > 0:
            recomendaciones.append(f"Recomiendo alimentar Yeso a MC3 (Espacio disponible: {esp_yeso:.2f}t)")

    if not recomendaciones:
        recomendaciones.append(f"Todas las tolvas de {molino_actual} están llenas o cerca de su capacidad máxima.")

    restricciones.append("Es necesario vaciar el sistema antes de una nueva alimentación para evitar contaminación.")
    restricciones.append("No se puede alimentar simultáneamente Clinker, Puzolana y Yeso a MC3.")
    restricciones.append("No se puede alimentar Clinker a múltiples molinos simultáneamente.")

    return recomendaciones, restricciones


def calcular_tiempo_vaciado(molino, material):
    if molino == "MC1":
        niveles = niveles_MC1
        tolvas = tolvas_MC1
    elif molino == "MC2":
        niveles = niveles_MC2
        tolvas = tolvas_MC2
    else:  # MC3
        niveles = niveles_MC3
        tolvas = tolvas_MC3
    
    # Manejar el caso especial de Puzolana para MC1 y MC3
    if material == "Puzolana" and molino in ["MC1", "MC3"]:
        capacidad = tolvas["Puzolana"]['capacidad']
        nivel_actual = niveles["Puzolana"]
    elif material == "Puzolana_Humeda" and molino == "MC2":
        capacidad = tolvas["Puzolana_Humeda"]['capacidad']
        nivel_actual = niveles["Puzolana_Humeda"]
    elif material == "Puzolana_Seca" and molino == "MC2":
        capacidad = tolvas["Puzolana_Seca"]['capacidad']
        nivel_actual = niveles["Puzolana_Seca"]
    else:
        capacidad = tolvas[material]['capacidad']
        nivel_actual = niveles[material]
    
    consumo = calcular_consumo(molino, material)
    
    if consumo > 0:
        return (nivel_actual - nivel_ineficiente(molino, material)) / consumo
    else:
        return float('inf')

def nivel_ineficiente(molino, material):
    if molino in ["MC1", "MC2"]:
        if molino == "MC1":
            return 0.2 * tolvas_MC1[material]['capacidad']  # 20% de la capacidad
        else:  # MC2
            if material in ["Puzolana_Humeda", "Puzolana_Seca"]:
                return 0.2 * tolvas_MC2[material]['capacidad']  # 20% de la capacidad
            else:
                return 0.2 * tolvas_MC2[material]['capacidad']  # 20% de la capacidad
    else:  # MC3
        return 0.5 * tolvas_MC3[material]['capacidad']  # 50% de la capacidad

def calcular_consumo(molino, material):
    tipo_produccion = obtener_tipo_produccion_actual(molino)
    necesidades = calcular_necesidades(molino, tipo_produccion)
    if molino == "MC2" and material in ["Puzolana_Humeda", "Puzolana_Seca"]:
        return necesidades.get("Puzolana_Humeda", 0) + necesidades.get("Puzolana_Seca", 0)
    return necesidades.get(material, 0)

def obtener_tipo_produccion_actual(molino):
    # Esta función debería obtener el tipo de producción actual del molino
    # Por ahora, usaremos un valor predeterminado
    if molino == "MC1":
        return "P30"
    elif molino == "MC2":
        return "P16"
    else:  # MC3
        return "P30"

def seleccionar_ruta_alimentacion(molino, material):
    if molino == "MC1":
        if material == "Clinker":
            return "Hacia MC1 desde Pretrit"
        elif material == "Puzolana":
            tiempo_vaciado_MC1 = calcular_tiempo_vaciado("MC1", "Puzolana")
            tiempo_vaciado_MC2 = calcular_tiempo_vaciado("MC2", "Puzolana_Humeda")
            return "Húmeda a MC1 por MC1" if tiempo_vaciado_MC1 < tiempo_vaciado_MC2 else "Húmeda a MC1 por MC2"
        elif material == "Yeso":
            tiempo_vaciado_MC1 = calcular_tiempo_vaciado("MC1", "Yeso")
            tiempo_vaciado_MC2 = calcular_tiempo_vaciado("MC2", "Yeso")
            return "Hacia MC1 por MC1" if tiempo_vaciado_MC1 < tiempo_vaciado_MC2 else "Hacia MC1 por MC2"
    elif molino == "MC2":
        if material == "Clinker":
            return "Hacia MC2 desde Pretrit"
        elif material == "Puzolana_Humeda":
            return "P.H. a 426HO04 por MC2"
        elif material == "Puzolana_Seca":
            return "P.S a 426HO02 por 426HO04"
        elif material == "Yeso":
            return "Hacia MC2 por MC2"
    elif molino == "MC3":
        if material == "Clinker":
            if niveles_MC3["Clinker_Silo_Blanco"] / tolvas_MC3["Clinker_Silo_Blanco"]["capacidad"] > 0.3:
                return "Hacia MC3 desde Silo3"
            else:
                return "Hacia Silo 3 desde Pretrit"
        elif material == "Puzolana":
            return "P. seca a MC3 por MC2"
        elif material == "Yeso":
            tiempo_vaciado_MC1 = calcular_tiempo_vaciado("MC1", "Yeso")
            tiempo_vaciado_MC2 = calcular_tiempo_vaciado("MC2", "Yeso")
            tiempo_vaciado_MC3 = calcular_tiempo_vaciado("MC3", "Yeso")
            if tiempo_vaciado_MC1 <= tiempo_vaciado_MC2 and tiempo_vaciado_MC1 <= tiempo_vaciado_MC3:
                return "Hacia MC3 por MC1"
            elif tiempo_vaciado_MC2 <= tiempo_vaciado_MC1 and tiempo_vaciado_MC2 <= tiempo_vaciado_MC3:
                return "Hacia MC3 por MC2"
            else:
                return "Hacia MC3 por MC3"
    return "Ruta no especificada"



def optimizar_alimentacion(molino, tipo_produccion):
    necesidades = calcular_necesidades(molino, tipo_produccion)
    print(f"\nNecesidades para {molino} - {tipo_produccion}:")
    for material, cantidad in necesidades.items():
        print(f"{material}: {cantidad:.2f} t/h")

    recomendaciones, restricciones = generar_recomendaciones(molino)
    print("\nRecomendaciones:")
    for recomendacion in recomendaciones:
        print(recomendacion)
    print("\nRestricciones:")
    for restriccion in restricciones:
        print(restriccion)

    for material, cantidad_necesaria in necesidades.items():
        if cantidad_necesaria > 0:
            ruta = seleccionar_ruta_alimentacion(molino, material)
            print(f"\nIntentando alimentar {material} a {molino} por la ruta: {ruta}")
            cantidad_alimentada = intentar_alimentar(molino, material, cantidad_necesaria)
            if cantidad_alimentada > 0:
                print(f"Se alimentó {cantidad_alimentada:.2f}t de {material} a {molino}")
            else:
                print(f"No se pudo alimentar {material} a {molino}")

def main():
    global niveles_MC1, niveles_MC2, niveles_MC3
    ciclo = 1

    while not todas_tolvas_llenas():
        print(f"\n--- Ciclo de optimización {ciclo} ---")
        molino = input("Ingrese el molino (MC1, MC2, MC3): ").upper()

        if molino not in ["MC1", "MC2", "MC3"]:
            print("Error, Ingrese un molino válido")
            continue

        if molino == "MC1":
            print("Este molino puede producir P30 o P40")
            tipos_validos = ["P30", "P40"]
        elif molino == "MC2":
            print("Este molino puede producir P10, P16, P20 o P30")
            tipos_validos = ["P10", "P16", "P20", "P30"]
        elif molino == "MC3":
            print("Este molino puede producir P30")
            tipos_validos = ["P30"]

        tipo_produccion = input("Ingrese el tipo de producción: ").upper()
        if tipo_produccion not in tipos_validos:
            print(f"Error, Ingrese un tipo de producción válido para {molino}")
            continue

        optimizar_alimentacion(molino, tipo_produccion)

        print("\nEstado actual después de la alimentación:")
        print(f"Niveles MC1: {redondear_diccionario(niveles_MC1)}")
        print(f"Niveles MC2: {redondear_diccionario(niveles_MC2)}")
        print(f"Niveles MC3: {redondear_diccionario(niveles_MC3)}")

        ciclo += 1

        continuar = input("¿Desea continuar con otro ciclo? (s/n): ").lower()
        if continuar != 's':
            break

    print("Simulación completada.")

if __name__ == "__main__":
    main()


--- Ciclo de optimización 1 ---


Error, Ingrese un molino válido

--- Ciclo de optimización 1 ---
Este molino puede producir P30 o P40

Necesidades para MC1 - P30:
Clinker: 49.32 t/h
Puzolana: 21.60 t/h
Yeso: 1.08 t/h

Recomendaciones:
Recomiendo alimentar Clinker a MC1 desde Pretrit (Espacio disponible: 400.00t)
Recomiendo alimentar Puzolana a MC1 (Espacio disponible: 240.00t)
Recomiendo alimentar Yeso a MC1 (Espacio disponible: 60.00t)

Restricciones:
Es necesario vaciar el sistema antes de una nueva alimentación para evitar contaminación.
No se puede alimentar simultáneamente Clinker, Puzolana y Yeso a MC3.
No se puede alimentar Clinker a múltiples molinos simultáneamente.

Intentando alimentar Clinker a MC1 por la ruta: Hacia MC1 desde Pretrit
Alimentado 49.32t de Clinker a MC1 (Nivel: 1.38m)
Se alimentó 49.32t de Clinker a MC1

Intentando alimentar Puzolana a MC1 por la ruta: Húmeda a MC1 por MC2
Por favor, ingrese un número válido.
Ajustando velocidad del transportador a 1111.00 RPM para puzolana
Alimentado 21.6